In [ ]:
# setting the Thesis project folder as working directory
%cd "../.."

In [ ]:
import random
import wandb
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import os
from datetime import datetime
from skimage import io

import sys
from utility_functions.utility import *

# Defining functions

Load and log

Things to check each run:
- project name
- artifact name
- artifact description

In [ ]:
def load_and_log(my_tensor_args, save_only, config, prj_name="Thesis"):
    '''
    This function relies on the definintion of 2 functions (my_tensor and my_3Dtensor)
    outside this one 
    '''

    # 🚀 start a run, with a type to label it and a project it can call home
    with wandb.init(project=prj_name, job_type="load-data") as run:

        print('Input shape to load_and_log: ', my_tensor_args['img'].shape)

        # check if we are passing already processed datasets
        if save_only:
            datasets = my_tensor_args['img'], my_tensor_args['annot']
        else:
            datasets = my_tensors(**my_tensor_args)  # separate code for loading the datasets
        names = ["images", "annotations"]

        # 🏺 create our Artifact
        raw_data = wandb.Artifact(
            "TR_img2-img3_eq", type="dataset",
            description="Validated annotations (3.2) by operator",
            metadata={"size_X": len(datasets[0]),
                      "size_Y": len(datasets[1]),
                      'sub-img_shape': my_tensor_args['img_size'],
                      'step_i_j': my_tensor_args['step_i_j'],
                      'signal_tresh': my_tensor_args['signal_treshold'],
                      'z_channel': config['z_channel'],
                      'rgb_channel': config['rgb_channel'],
                      'original_img_shape': my_tensor_args['img'].shape,
                      'min_max': (np.min(my_tensor_args['img']), np.max(my_tensor_args['img'])),
                      'Whole_image_dataset': config['dataset_WI'],
                      'y_sum_treshold': my_tensor_args['Y_sum_tresh'],
                      'OTHER': config['other'],
                      })

        for name, data in zip(names, datasets):
            # 🐣 Store a new file in the artifact, and write something into its contents.
            with raw_data.new_file(name + ".npy", mode="wb") as file:
                np.save(file, data)

        # ✍️ Save the artifact to W&B.
        run.log_artifact(raw_data)

# Setting default parameters and seeds

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# plots sizes
plt.rcParams['figure.figsize'] = [11, 11]

config

In [ ]:
config = {
    'dataset_WI': 'white-mad-hatters/Team-Artifacts-Tesi/GBC-2images-EqAdapthist-raw:v4',
    'img_size': 128,
    'step_ij': 128,
    'sign_treshold': 0,
    'Y_sum_tresh': 0,
    'z_channel': (0, 1, 2, 3),
    'rgb_channel': (0, 1), #, 1
    'other': None
    
}

In [ ]:
wandb.login()

# Image loading and cropping

In [ ]:
X, Y = download_data('Team-Artifacts-Tesi', config['dataset_WI'], entity='white-mad-hatters', ann_filename='annot2.npy', img_filename='green_blue_fusion2.npy')

In [ ]:
X.shape, Y.shape, (np.min(X[:, :, 0:2]), np.max(X[:, :, 0:2])), np.unique(Y)

In [ ]:
plt.imshow(X[:, :, 0])

In [ ]:
np.sum(X.flatten()==-1)

# Create final dataset

In [ ]:
row_start = 0
row_end = int(X.shape[0]/2 + config['step_ij']/2)  #len(X)
config['other'] ={"row_start": row_start,
                    'row_end': row_end}
row_start, row_end

In [ ]:
my_tensor_args = dict(img=X[row_start:row_end, :, 0:2],
                      annot=Y,
                      img_size=config['img_size'],
                      step_i_j=config['step_ij'],
                      signal_treshold=config['sign_treshold'],
                      Y_sum_tresh=config['Y_sum_tresh']
                      )

img2 train dataset

In [ ]:
X1, Y1 = my_tensors(**my_tensor_args)

In [ ]:
print(X1.shape, Y1.shape)

img3 train dataset

In [ ]:
X2, Y2 = my_tensors(**my_tensor_args)

In [ ]:
print(X2.shape, Y2.shape)

merge 2 training datasets

In [ ]:
X = np.concatenate((X1, X2), axis=0, dtype=np.float32)
Y = np.concatenate((Y1, Y2), axis=0, dtype=np.float32)

In [ ]:
X.shape, Y.shape

# Create and Load (.npy) to W&B

To check:
- img
- annot

In [ ]:
my_tensor_args = dict(img=X,
                      annot=Y,
                      img_size=config['img_size'],
                      step_i_j=config['step_ij'],
                      signal_treshold=config['sign_treshold'],
                      Y_sum_tresh=config['Y_sum_tresh']
                      )

To check:
- save_only parameter
- whole image from which data are coming from

In [ ]:
load_and_log(my_tensor_args, save_only=True, config=config,)